In [1]:
import numpy
import scipy
import pandas
import sklearn
from sklearn.linear_model import LogisticRegression
import csv
import random
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn import mixture
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,GradientBoostingClassifier
from sklearn import tree

dataIn = 'gameScore_rates.csv'
df = pandas.read_csv(dataIn)
grouped = df['homeWin'].groupby(df['game_year'])
print (grouped.mean())
df.columns

game_year
2008    0.608838
2009    0.594309
2010    0.604065
2011    0.585859
2012    0.611880
2013    0.580488
Name: homeWin, dtype: float64


Index(['match_id', 'game_year', 'home_team', 'home_team_score', 'visit_team',
       'visit_team_score', 'RPM_weight_0', 'ORPM_weight_0', 'DRPM_weight_0',
       'PER_weight_0', 'RPM_weight_1', 'ORPM_weight_1', 'DRPM_weight_1',
       'PER_weight_1', 'avg_scoreDiff', 'avg_homeWin', 'avg_scoreDiff_home',
       'avg_win_home', 'avg_scoreDiff_visit', 'avg_win_visit', 'homeWin'],
      dtype='object')

In [2]:
df.head()

,match_id,game_year,home_team,home_team_score,visit_team,visit_team_score,RPM_weight_0,ORPM_weight_0,DRPM_weight_0,PER_weight_0,...,ORPM_weight_1,DRPM_weight_1,PER_weight_1,avg_scoreDiff,avg_homeWin,avg_scoreDiff_home,avg_win_home,avg_scoreDiff_visit,avg_win_visit,homeWin
0,311225018,2011,ny,106,bos,104,0.653622,-0.284343,0.888224,14.744700,...,0.653205,0.182854,17.064548,-6.0,0.0,1.902439,0.560976,2.560976,0.560976,1
1,311227014,2011,mia,115,bos,107,0.653622,-0.284343,0.888224,14.744700,...,1.147897,1.046426,17.020164,9.0,0.5,9.097561,0.731707,2.560976,0.560976,1
2,311228003,2011,no,97,bos,78,0.750899,-0.291455,0.988420,14.470200,...,-1.441477,0.116085,12.984575,-4.0,0.0,3.560976,0.682927,2.560976,0.560976,1
3,311230002,2011,bos,96,det,85,-0.635635,0.026043,-0.663022,14.466331,...,0.036796,1.090354,15.219371,7.5,1.0,8.170732,0.804878,-6.731707,0.219512,1
4,320101027,2011,wsh,86,bos,94,1.586591,0.278886,1.264432,15.762429,...,-0.814253,-0.200495,13.972442,1.5,1.0,-2.048780,0.487805,2.560976,0.560976,0


In [5]:
xColumns = ['RPM_weight_0', 'ORPM_weight_0', 'DRPM_weight_0', 'PER_weight_0', 
            'RPM_weight_1', 'ORPM_weight_1', 'DRPM_weight_1', 'PER_weight_1']
xColumnsRPM = ['RPM_weight_0', 'ORPM_weight_0', 'DRPM_weight_0', 'RPM_weight_1', 'ORPM_weight_1', 'DRPM_weight_1']
xColumnsPER = ['PER_weight_0', 'PER_weight_1']

def scale(df):
    tmp = pandas.DataFrame(df)
    for col in tmp.columns:
        if tmp[col].min() <0:
            data = abs(tmp[col].min())
            tmp[col] = tmp[col].apply(lambda x: x+data)
            maxV = float(tmp[col].max())
            if maxV ==0:
                continue
            tmp[col] = tmp[col].apply(lambda x: x/maxV)
    return tmp

In [6]:
#logistic. best so far
for year in range(2009,2014):
    train = df[df['game_year']<year]
    test = df[df['game_year']==year]
    xTrain = scale(train[xColumns])
    yTrain = train['homeWin']
    xTest = scale((test[xColumns]))
    yTest = test['homeWin']
    logreg = LogisticRegression()
    logreg.fit(xTrain, yTrain)
    acc = logreg.score(xTest,yTest)
    print('Accuray for '+str(year)+': ',acc)

Accuray for 2009:  0.656910569106
Accuray for 2010:  0.684552845528
Accuray for 2011:  0.660606060606
Accuray for 2012:  0.684296175753
Accuray for 2013:  0.639837398374


In [17]:
#compare all for each year
naive_bayes = GaussianNB()
treeClf = tree.DecisionTreeClassifier()
extraTtreeClf = tree.ExtraTreeClassifier()
randomTreeClf = RandomForestClassifier(n_estimators=100)
extraTreeClf = ExtraTreesClassifier(n_estimators=100)
gBoostTreeClf = GradientBoostingClassifier(n_estimators=100)
lassoClf = sklearn.linear_model.Lasso()
logiClf = LogisticRegression()
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=1.0)
lin_svc = svm.LinearSVC(C=1.0)
for clf in [treeClf,extraTtreeClf,randomTreeClf,extraTreeClf,gBoostTreeClf,lassoClf,logiClf,rbf_svc, lin_svc, naive_bayes]:
    for year in range(2009,2014):
        train = df[df['game_year']<year]
        test = df[df['game_year']==year]
        xTrain = scale(train[xColumnsRPM])
        yTrain = train['homeWin']
        yTest = test['homeWin']
        xTest = scale((test[xColumnsRPM]))
        model = clf.fit(xTrain, yTrain)
        acc = model.score(xTest,yTest)
        model_name = str(clf).split('(')[0]
        print ('Accuray for '+str(year)+' with ' + model_name+': ',acc)

Accuray for 2009 with DecisionTreeClassifier:  0.567479674797
Accuray for 2010 with DecisionTreeClassifier:  0.482113821138
Accuray for 2011 with DecisionTreeClassifier:  0.560606060606
Accuray for 2012 with DecisionTreeClassifier:  0.565500406835
Accuray for 2013 with DecisionTreeClassifier:  0.557723577236
Accuray for 2009 with ExtraTreeClassifier:  0.566666666667
Accuray for 2010 with ExtraTreeClassifier:  0.542276422764
Accuray for 2011 with ExtraTreeClassifier:  0.587878787879
Accuray for 2012 with ExtraTreeClassifier:  0.564686737185
Accuray for 2013 with ExtraTreeClassifier:  0.565040650407
Accuray for 2009 with RandomForestClassifier:  0.621138211382
Accuray for 2010 with RandomForestClassifier:  0.547154471545
Accuray for 2011 with RandomForestClassifier:  0.626262626263
Accuray for 2012 with RandomForestClassifier:  0.633848657445
Accuray for 2013 with RandomForestClassifier:  0.60162601626
Accuray for 2009 with ExtraTreesClassifier:  0.629268292683
Accuray for 2010 with Extr